In [1]:
import logging
from dataclasses import dataclass
from typing import Union

import torch
from torch.distributions import Dirichlet

logger = logging.getLogger(__name__)

In [3]:
import numpy as np
nb_classes = int(2e4)
alpha = 1e-3
for i in range(100):
    tmp = Dirichlet(concentration = torch.tensor([alpha, ] * nb_classes)).sample((20,)).max()
    if np.abs(tmp.max().item() - 1) > .1:
        print(tmp.max())

tensor(0.2146)
tensor(0.2272)
tensor(0.2811)
tensor(0.1891)


tensor(0.2833)
tensor(0.2067)
tensor(0.2339)
tensor(0.2494)
tensor(0.2236)
tensor(0.1880)
tensor(0.2136)
tensor(0.2392)
tensor(0.1730)
tensor(0.1848)
tensor(0.3161)
tensor(0.2534)
tensor(0.1773)
tensor(0.2346)
tensor(0.2218)
tensor(0.2208)
tensor(0.1536)
tensor(0.1981)
tensor(0.1817)
tensor(0.2015)
tensor(0.1841)
tensor(0.1945)
tensor(0.1786)
tensor(0.2205)
tensor(0.2161)
tensor(0.2053)
tensor(0.2318)
tensor(0.1715)
tensor(0.2262)
tensor(0.2172)
tensor(0.1974)
tensor(0.1677)
tensor(0.1969)
tensor(0.1728)
tensor(0.1558)
tensor(0.2152)
tensor(0.2279)
tensor(0.1759)
tensor(0.1630)
tensor(0.2206)
tensor(0.1742)
tensor(0.2363)
tensor(0.2471)
tensor(0.2165)
tensor(0.1672)
tensor(0.3536)
tensor(0.1785)
tensor(0.1901)
tensor(0.2034)
tensor(0.2586)
tensor(0.1960)
tensor(0.1993)
tensor(0.2109)
tensor(0.2006)
tensor(0.1588)
tensor(0.2650)
tensor(0.2455)
tensor(0.2446)
tensor(0.1970)
tensor(0.2884)
tensor(0.2127)
tensor(0.1673)
tensor(0.2186)
tensor(0.1936)
tensor(0.2053)
tensor(0.2672)
tensor(0.1

In [5]:
tmp

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]])

In [132]:
nb_input = 2**10
factor_emb_dim = 10
input_factors = [3, 4, 2, 4, 1]
output_factors = [2, 3, 1, 1, 1]
alphas = [1e-3, 1e-3, 1e-3, 1e-3, 1e-3]
alphas = [100., ] * 5

all_emb = torch.randn((nb_input, factor_emb_dim))


p_y_x = torch.ones((nb_input, *output_factors))
view = [nb_input] + [1, ] * len(input_factors)

for i, (alpha, input_factor, output_factor) in enumerate(zip(alphas, input_factors, output_factors)):
    directions = torch.randn((factor_emb_dim, input_factor))
    sign = (all_emb @ directions).sign()
    value = (torch.tensor([2 ** i for i in range(input_factor)]) * sign).sum(axis=1)

    p_yi = Dirichlet(concentration = torch.tensor([alpha, ] * output_factor)).sample((2 ** input_factor, ))
    p_yis_x = p_yi[value.long()]

    view[i+1] = output_factor
    p_y_x *= p_yis_x.view(view)
    view[i+1] = 1

p_y_x = p_y_x.view(nb_input, -1)

In [134]:
p_y_x.sum(axis=1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000])

In [94]:
p_y_x.shape, p_yis_x.shape

(torch.Size([1024, 2, 3, 1, 1, 1]), torch.Size([1024, 1]))

In [30]:
torch.tensor([2 ** i for i in range(size_factor)])

tensor([1, 2, 4])

In [21]:
zx = [(torch.tensor([2 ** i for i in range(U.shape[1])]) * (torch.sign(x @ U) + 1) / 2).sum(axis=1) for U in Us]

torch.Size([1024])

In [28]:
torch.eye(2**3)[zx[0].to(int)]

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
for p_yi in p_yi_list[1:]:
    p_y = p_y[..., np.newaxis] * p_yi 

In [25]:
import numpy as np

def joint_probability_numpy(p_yi_list):
    """
    Computes the joint probability distribution p(y) as the product of p(y_i).
    
    Args:
        p_yi_list (list of np.ndarray): List of probability vectors for each discrete factor y_i.
    
    Returns:
        np.ndarray: Joint probability distribution vector p(y).
    """
    # Ensure the list is not empty
    if not p_yi_list:
        raise ValueError("The list of probability vectors cannot be empty.")
    
    # Start with the first probability vector
    p_y = p_yi_list[0]
    
    # Multiply the probability vectors element-wise
    for p_yi in p_yi_list[1:]:
        p_y = p_y[..., np.newaxis] * p_yi  # Broadcasting for multiplication
    
    # Flatten the result to get a 1D joint probability distribution
    return p_y.flatten()

# Example usage
p_y1 = np.array([0.5, 0.5])  # p(y1)
p_y2 = np.array([0.3, 0.7])  # p(y2)
p_y3 = np.array([0.2, 0.8])  # p(y3)

joint_p = joint_probability_numpy([p_y1, p_y2, p_y3])
print("Joint Probability Distribution:", joint_p)


Joint Probability Distribution: [0.03 0.12 0.07 0.28 0.03 0.12 0.07 0.28]
